# Agilent 34411A versus Keysight 34465A

The following notebook perfoms a benchmarking of the two DMMs. In part one, raw readings of immediate voltages are timed
and compared. In part two, actual sweeps are performed with a QDac.

In [1]:
%matplotlib notebook
import time
import matplotlib.pyplot as plt
import numpy as np

import qcodes as qc
from qcodes.instrument_drivers.Keysight.Keysight_34465A import Keysight_34465A
from qcodes.instrument_drivers.agilent.Agilent_34400A import Agilent_34400A
from qcodes.instrument_drivers.QDev.QDac import QDac

## Import and setup

In [2]:
ks = Keysight_34465A('Keysight', 'TCPIP0::K-000000-00000::inst0::INSTR')
agi = Agilent_34400A('Agilent', 'TCPIP0::192.168.15.105::inst0::INSTR')
qdac = QDac('qdac', 'ASRL4::INSTR')

Connected to: Keysight Technologies 34465A (serial:MY54505388, firmware:A.02.14-02.40-02.14-00.49-02-01) in 6.93s
Connected to: Agilent Technologies 34411A (serial:MY48002016, firmware:2.35-2.35-0.09-46-09) in 0.06s


In [9]:
station = qc.Station(ks, agi, qdac)

In [10]:
# set up a NPLC of 0.06 corresponding to an aperture time of 1 ms
ks.NPLC(0.06)
agi.NPLC(0.06)

# set the same range on both DMMs
ks.range(1)
agi.range(1)

## Part one - raw readings

In [11]:
# Preliminary testing, just get N readings from each instrument
# with the displays on

ks.display_clear()  # make sure that the display is on
agi.display_clear()

print('Reading with displays ON')
print('-'*10)

for N in [512, 1024, 2048]:

    t_start = time.time()
    for ii in range(N):
        ks.volt()
    t_stop = time.time()
    print('Read {} values one at a time from Keysight in: {} s'.format(N, t_stop-t_start))

    t_start = time.time()
    for ii in range(N):
        agi.volt()
    t_stop = time.time()
    print('Read {} values one at a time from Agilent in: {} s'.format(N, t_stop-t_start))

    print('-'*10)

Reading with displays ON
----------
Read 512 values one at a time from Keysight in: 8.587134838104248 s
Read 512 values one at a time from Agilent in: 4.099916696548462 s
----------
Read 1024 values one at a time from Keysight in: 16.83395004272461 s
Read 1024 values one at a time from Agilent in: 8.229825019836426 s
----------
Read 2048 values one at a time from Keysight in: 33.76206350326538 s
Read 2048 values one at a time from Agilent in: 16.453675985336304 s
----------


In [12]:
# The same test, but with the displays off to decrease latency

print('Reading with displays OFF')
print('-'*10)

for N in [512, 1024, 2048]:

    ks.display_text('QCoDeS')
    t_start = time.time()
    for ii in range(N):
        ks.volt()
    t_stop = time.time()
    print('Read {} values one at a time from Keysight in: {} s'.format(N, t_stop-t_start))
    ks.display_clear()

    agi.display_text('QCoDes')
    t_start = time.time()
    for ii in range(N):
        agi.volt()
    t_stop = time.time()
    print('Read {} values one at a time from Agilent in: {} s'.format(N, t_stop-t_start))
    agi.display_clear()

    print('-'*10)

Reading with displays OFF
----------
Read 512 values one at a time from Keysight in: 4.5132529735565186 s
Read 512 values one at a time from Agilent in: 4.098856449127197 s
----------
Read 1024 values one at a time from Keysight in: 8.996399164199829 s
Read 1024 values one at a time from Agilent in: 8.227899074554443 s
----------
Read 2048 values one at a time from Keysight in: 17.969755172729492 s
Read 2048 values one at a time from Agilent in: 16.399696350097656 s
----------


## Part two - QCoDeS looping

### 1D Sweep

In [13]:
# Sweep a voltage from 0.2 V to 0.5 V in 512 steps
# Switch off displays for speed

ks.display_text('QCoDeS')
agi.display_text('QCoDeS')

N = 512  
V1 = 0.2
V2 = 0.5
dV = (V2-V1)/(N-1)  # endpoint included in sweep

loop = qc.Loop(qdac.ch42_v.sweep(V1, V2, dV)).each(ks.volt)
data = loop.get_data_set(name='testsweep')
t_start = time.time()
_ = loop.run()
t_stop = time.time()
print('\n\nDid a {}-point QCoDeS sweep of the QDac/Keysight pair in {:.1f} s\n\n'.format(N, t_stop-t_start))

loop = qc.Loop(qdac.ch41_v.sweep(V1, V2, dV)).each(agi.volt)
data = loop.get_data_set(name='testsweep')
t_start = time.time()
_ = loop.run()
t_stop = time.time()
print('\n\nDid a {}-point QCoDeS sweep of the QDac/Agilent pair in {:.1f} s\n\n'.format(N, t_stop-t_start))

agi.display_clear()
ks.display_clear()

DataSet:
   mode     = DataMode.LOCAL
   location = 'data/2017-03-07/#018_testsweep_12-32-13'
   <Type>   | <array_id>      | <array.name> | <array.shape>
   Setpoint | qdac_ch42_v_set | ch42_v       | (512,)
   Measured | Keysight_volt   | volt         | (512,)
started at 2017-03-07 12:32:29


Did a 512-point QCoDeS sweep of the QDac/Keysight pair in 16.5 s


DataSet:
   mode     = DataMode.LOCAL
   location = 'data/2017-03-07/#019_testsweep_12-32-29'
   <Type>   | <array_id>      | <array.name> | <array.shape>
   Setpoint | qdac_ch41_v_set | ch41_v       | (512,)
   Measured | Agilent_volt    | volt         | (512,)
started at 2017-03-07 12:32:46


Did a 512-point QCoDeS sweep of the QDac/Agilent pair in 16.5 s




### 2D Sweep

In [14]:
# Perform the same sweep as before, but this time nested inside another sweep of the same length,
# i.e. at each point of the original sweep, another QDac channel sweeps its voltage in the same number of steps

ks.display_text('QCoDeS')
agi.display_text('QCoDeS')

N = 100
V1 = 0.2
V2 = 0.5
dV = (V2-V1)/(N-1)

loop = qc.Loop(qdac.ch42_v.sweep(V1, V2, dV)).loop(qdac.ch41_v.sweep(V1, V2, dV)).each(ks.volt)
data = loop.get_data_set(name='testsweep')
t_start = time.time()
_ = loop.run()
t_stop = time.time()
print('\n\nDid a {}x{}-point QCoDeS sweep of the QDac/Keysight pair in {:.1f} s\n\n'.format(N, N, t_stop-t_start))

loop = qc.Loop(qdac.ch41_v.sweep(V1, V2, dV)).loop(qdac.ch42_v.sweep(V1, V2, dV)).each(agi.volt)
data = loop.get_data_set(name='testsweep')
t_start = time.time()
_ = loop.run()
t_stop = time.time()
print('\n\nDid a {}x{}-point QCoDeS sweep of the QDac/Agilent pair in {:.1f} s\n\n'.format(N, N, t_stop-t_start))

ks.display_clear()
agi.display_clear()

DataSet:
   mode     = DataMode.LOCAL
   location = 'data/2017-03-07/#020_testsweep_12-33-46'
   <Type>   | <array_id>      | <array.name> | <array.shape>
   Setpoint | qdac_ch42_v_set | ch42_v       | (100,)
   Setpoint | qdac_ch41_v_set | ch41_v       | (100, 100)
   Measured | Keysight_volt   | volt         | (100, 100)
started at 2017-03-07 12:39:09


Did a 100x100-point QCoDeS sweep of the QDac/Keysight pair in 323.9 s


DataSet:
   mode     = DataMode.LOCAL
   location = 'data/2017-03-07/#021_testsweep_12-39-09'
   <Type>   | <array_id>      | <array.name> | <array.shape>
   Setpoint | qdac_ch41_v_set | ch41_v       | (100,)
   Setpoint | qdac_ch42_v_set | ch42_v       | (100, 100)
   Measured | Agilent_volt    | volt         | (100, 100)
started at 2017-03-07 12:44:33


Did a 100x100-point QCoDeS sweep of the QDac/Agilent pair in 323.8 s




In [14]:
ks.close()
agi.close()
qdac.close()